# Example EDSL 
This notebook provides example code shown in the [teaching guide](https://docs.expectedparrot.com/en/latest/teaching.html).

Before running the code below please see instructions on [getting started](https://www.expectedparrot.com/getting-started) using EDSL.

## Create a survey

In [1]:
from edsl import QuestionMultipleChoice, QuestionNumerical, Survey

# Create questions
q1 = QuestionMultipleChoice(
    question_name="preferred_source",
    question_text="What is your preferred source of national news?", 
    question_options=["television", "social media", "online news", "newspaper", "radio", "podcast", "other"],
)
q2 = QuestionNumerical(
    question_name="hours_per_week",
    question_text="How many hours per week on average do you consume national news via {{ preferred_source.answer }}?" # piping answer from previous question
)

# Create a survey
survey = Survey(questions = [q1, q2])

## Run it with AI agents and language models

In [2]:
from edsl import Agent, AgentList, Model, ModelList

# Create AI agents
a = AgentList(Agent(traits = {"persona":p}) for p in ["college student", "retired professional"])

# Select language models
m = ModelList([
    Model("gpt-4o", service_name = "openai"),
    Model("claude-3-sonnet-20240229", service_name = "anthropic")
])

# Run the survey with the agent and model
results = survey.by(a).by(m).run()

## Inspect results
Results are accessible at [Coop](https://www.expectedparrot.com/content/explore) at the above link.
You can also access them in your notebook:

In [3]:
# To see all columns of the results
results.columns 

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,agent.persona
4,answer.hours_per_week
5,answer.preferred_source
6,cache_keys.hours_per_week_cache_key
7,cache_keys.preferred_source_cache_key
8,cache_used.hours_per_week_cache_used
9,cache_used.preferred_source_cache_used


## Analyze results
Learn about methods for analyzing [results](https://docs.expectedparrot.com/en/latest/results.html).

In [4]:
# Inspect components of results
results.select("model", "persona", "preferred_source", "hours_per_week")

,model.model,agent.persona,answer.preferred_source,answer.hours_per_week
0,gpt-4o,college student,online news,3.500000
1,claude-3-sonnet-20240229,college student,online news,5.000000
2,gpt-4o,retired professional,newspaper,2.500000
3,claude-3-sonnet-20240229,retired professional,online news,10.000000


We can filter, sort, use SQL queries, etc. For example:

In [5]:
(
    results
        .filter("{{ model.model }} == 'gpt-4o' and {{ agent.persona }} == 'college student'")
        .sort_by("hours_per_week")
        .select("model", "persona", "preferred_source", "hours_per_week", "hours_per_week_comment")
)

,model.model,agent.persona,answer.preferred_source,answer.hours_per_week,comment.hours_per_week_comment
0,gpt-4o,college student,online news,3.500000,"I usually check the news online between classes or when I'm taking a break from studying, so it adds up to a few hours a week."


In [6]:
results.sql("""
select 
    model, 
    persona, 
    preferred_source, 
    preferred_source_comment
from self
where 1=1 
    and persona == 'retired professional'
    and model == 'claude-3-sonnet-20240229'
""")

,model,persona,preferred_source,preferred_source_comment
0,claude-3-sonnet-20240229,retired professional,online news,"# As a retired professional, I prefer online news sources as they allow me to easily access a wide variety of reputable national and international news outlets at my convenience. Online news is also frequently updated throughout the day."


## Launch with humans
Here we generate a [web-based version](https://docs.expectedparrot.com/en/latest/humanize.html) of the survey and import the responses:

In [7]:
info = survey.humanize()
info

{'project_name': 'Project',
 'uuid': 'dc4fe30d-5101-468c-9256-e88da78d80ec',
 'admin_url': 'https://www.expectedparrot.com/home/projects/dc4fe30d-5101-468c-9256-e88da78d80ec',
 'respondent_url': 'https://www.expectedparrot.com/respond/dc4fe30d-5101-468c-9256-e88da78d80ec'}

In [8]:
from edsl import Coop

coop = Coop()

human_results = coop.get_project_human_responses(info["uuid"])
human_results

,hours_per_week,preferred_source,scenario_index,agent_name,agent_index,agent_instruction,inference_service,temperature,model_index,model,hours_per_week_system_prompt,hours_per_week_user_prompt,preferred_source_user_prompt,preferred_source_system_prompt,preferred_source_raw_model_response,hours_per_week_input_tokens,preferred_source_input_tokens,hours_per_week_raw_model_response,preferred_source_output_price_per_million_tokens,hours_per_week_input_price_per_million_tokens,hours_per_week_cost,hours_per_week_output_price_per_million_tokens,preferred_source_one_usd_buys,hours_per_week_one_usd_buys,preferred_source_input_price_per_million_tokens,preferred_source_output_tokens,hours_per_week_output_tokens,preferred_source_cost,iteration,hours_per_week_question_text,preferred_source_question_text,hours_per_week_question_options,preferred_source_question_options,hours_per_week_question_type,preferred_source_question_type,preferred_source_comment,hours_per_week_comment,hours_per_week_generated_tokens,preferred_source_generated_tokens,preferred_source_cache_used,hours_per_week_cache_used,hours_per_week_cache_key,preferred_source_cache_key,hours_per_week_reasoning_summary,preferred_source_reasoning_summary
0,7,newspaper,0,eac139b7-0610-4e5d-afff-3716488ba2bc,0,nan,test,0.500000,0,test,nan,nan,nan,nan,Not Applicable,nan,nan,Not Applicable,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,How many hours per week on average do you consume national news via {{ preferred_source.answer }}?,What is your preferred source of national news?,nan,"['television', 'social media', 'online news', 'newspaper', 'radio', 'podcast', 'other']",numerical,multiple_choice,This is a real survey response from a human.,This is a real survey response from a human.,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,nan,nan
1,5,social media,0,d2d69be6-71c2-472d-bb6f-5f0a565b284e,1,nan,test,0.500000,0,test,nan,nan,nan,nan,Not Applicable,nan,nan,Not Applicable,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,How many hours per week on average do you consume national news via {{ preferred_source.answer }}?,What is your preferred source of national news?,nan,"['television', 'social media', 'online news', 'newspaper', 'radio', 'podcast', 'other']",numerical,multiple_choice,This is a real survey response from a human.,This is a real survey response from a human.,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,nan,nan


In [9]:
combined_results = results + human_results

combined_results.select("model", "agent_name", "preferred_source", "hours_per_week")

,model.model,agent.agent_name,answer.preferred_source,answer.hours_per_week
0,gpt-4o,Agent_0,online news,3.500000
1,claude-3-sonnet-20240229,Agent_1,online news,5.000000
2,gpt-4o,Agent_2,newspaper,2.500000
3,claude-3-sonnet-20240229,Agent_3,online news,10.000000
4,test,eac139b7-0610-4e5d-afff-3716488ba2bc,newspaper,7.000000
5,test,d2d69be6-71c2-472d-bb6f-5f0a565b284e,social media,5.000000


You can also launch your survey with participants at Prolific ([learn more](https://docs.expectedparrot.com/en/latest/prolific.html)).

## Posting to Coop
The above results were automatically stored at [Coop](https://docs.expectedparrot.com/en/latest/coop.html).
We can also manually post any EDSL object using the `push()` method.
Here we post this notebook:

In [10]:
# from edsl import Notebook

# nb = Notebook("example_edsl_teaching.ipynb")

# nb.push(
#     description = "Example EDSL code",
#     alias = "example-edsl-teaching",
#     visibility = "public"
# )